In [1]:
from ruptura.RupturaTable import RupturaTable
from ruptura.CreateBatch import CreateBatch
import collections

In [2]:
rupturaTable = RupturaTable()
files = ['barbieri-only.csv']  #concat all files here
clipData = ['01/11/2018', '01/04/2019']
rupturaTable.generate(files,clipData)

In [3]:
%%time
createBatch = CreateBatch('0-1-0')
amostras = createBatch.create(rupturaTable.getData(), 'BARBIERI', '01/03/2019')
createBatch.exportBatch(amostras, 'barbieri-ymod.json')

Wall time: 21.8 s


In [4]:
amostras['Mart Minas Paracatu Loja 224-BISCOITO BARBIERI ESCALDADO 150G']

{'x': [[0, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [0.33, 0, 0],
  [1, 0, 0],
  [0, 0, 1],
  [0, 0, 1],
  [0.33, 0, 0],
  [0, 0, 1],
  [0.33, 0, 0],
  [0, 0, 1],
  [0, 1, 0],
  [0, 0, 1],
  [0, 0, 1],
  [0.33, 0, 0],
  [0, 0, 1],
  [0.33, 0, 0],
  [0, 0, 1],
  [0.33, 0, 0],
  [0, 0, 1],
  [0, 0, 1],
  [0.33, 0, 0],
  [0, 0, 1],
  [0.33, 0, 0],
  [0.33, 0, 0],
  [0, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [1, 0, 0],
  [0, 0, 1],
  [0, 0, 1],
  [1, 0, 0],
  [0, 0, 1],
  [0, 0, 1],
  [0.33, 0, 0],
  [1, 0, 0],
  [0, 0, 1],
  [0, 0, 1],
  [0.33, 0, 0],
  [0, 0, 1],
  [1, 0, 0],
  [0, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [0, 0, 1],
  [0.33, 0, 0],
  [0, 0, 1],
  [0.33, 0, 0],
  [0, 0, 1],
  [0, 0, 1],
  [0.33, 0, 0],
  [0, 0, 1],
  [0.33, 0, 0]],
 'y': [[0, 1],
  [0, 1],
  [0, 1],
  [0.33, 0],
  [1, 0],
  [0, 1],
  [0, 1],
  [0.33, 0],
  [0, 1],
  [0.33, 0],
  [0, 1],
  [0, 0],
  [0, 1],
  [0, 1],
  [0.33, 0],
  

# GERANDO TABELA PARA CONFERENCIA

In [ ]:
# APENAS BARBIERI
indBarbieri = []
for i in data.index:
    if data.Produto[i] in todosBiscoitos:
        indBarbieri.append(i)
dataBarbieri = data.iloc[indBarbieri,:].copy()
dataBarbieri = dataBarbieri.reset_index(drop=True)
dataBarbieri.to_csv('data-barbieri.csv',sep=';')